In [ ]:
import tomotopy as tp
print(tp.isa) # prints 'avx2', 'avx', 'sse2' or 'none'

In [ ]:
from tomotopy import DMRModel
from tomotopy.utils import Corpus

In [ ]:
import tomotopy as tp
import numpy as np
import nltk
import pyLDAvis

In [ ]:
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora, models, similarities
from string import punctuation
import numpy
import os
import json
import datetime
import signal
import re
import string
import tomotopy as tp
import numpy as np
import nltk
import pyLDAvis
from gensim.matutils import hellinger
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#stopwords - change stopword list - TODO
from nltk.tokenize import TweetTokenizer

def preprocessing(text):
	negate_dict = {
    "couldn't": "could not",
	"can't": "can not",
    "didn't": "did not",
    "won't": "will not",
    "don't": "do not",
    "aren't": "are not",
    "doesn't": "does not",
    "hadn't": "had not", 
    "hasn't": "has not",
    "haven't": "have not",
    "isn't": "is not",
    "mightn't": "might not",
    "mustn't": "must not",
    "needn't": "need not",
    "shan't": "shall not",
    "shouldn't": "should not",
    "wasn't": "was not",
    "weren't": "were not",
    "wouldn't": "would not"
    }
	stemmer = PorterStemmer()
	stopwords_english = stopwords.words('english')
	stopwords_english.remove('not')
	stopwords_english.remove('no')
	text = re.sub(r'$', '', str(text))
	text = re.sub(r'https?:\/\/.*[\r\n]*', '', str(text))
	text = re.sub(r'^RT[\s]+', '', str(text))
	text = re.sub(r'#', '', str(text))
	text = re.sub(r'\@\w*', '', str(text))
	text = re.sub(r'WHO', 'world health organization', str(text))
	for negate_word in negate_dict.keys():
		text = re.sub(negate_word, negate_dict[negate_word], str(text))
	text = re.sub(r"&", ' and ', str(text))
	text = text.replace('&amp', ' ')
	text = text.replace('amp', ' ')
	text = re.sub(r"[^0-9a-zA-Z]+", ' ', str(text))
	tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
	text_tokens = tokenizer.tokenize(text)
	clean_text = []
	for word in text_tokens:
		if (word not in stopwords_english and word not in string.punctuation):
			stem_word = stemmer.stem(word)
			clean_text.append(stem_word)
	return ' '.join(clean_text)

def preprocessing_to_token(text):
	negate_dict = {
    "couldn't": "could not",
	"can't": "can not",
    "didn't": "did not",
    "won't": "will not",
    "don't": "do not",
    "aren't": "are not",
    "doesn't": "does not",
    "hadn't": "had not", 
    "hasn't": "has not",
    "haven't": "have not",
    "isn't": "is not",
    "mightn't": "might not",
    "mustn't": "must not",
    "needn't": "need not",
    "shan't": "shall not",
    "shouldn't": "should not",
    "wasn't": "was not",
    "weren't": "were not",
    "wouldn't": "would not"
    }
	stemmer = PorterStemmer()
	stopwords_english = stopwords.words('english')
	stopwords_english.remove('not')
	stopwords_english.remove('no')
	text = re.sub(r'$', '', str(text))
	text = re.sub(r'https?:\/\/.*[\r\n]*', '', str(text))
	text = re.sub(r'^RT[\s]+', '', str(text))
	text = re.sub(r'#', '', str(text))
	text = re.sub(r'\@\w*', '', str(text))
	text = re.sub(r'WHO', 'world health organization', str(text))
	for negate_word in negate_dict.keys():
		text = re.sub(negate_word, negate_dict[negate_word], str(text))
	text = re.sub(r"&", ' and ', str(text))
	text = text.replace('&amp', ' ')
	text = text.replace('amp', ' ')
	text = re.sub(r"[^0-9a-zA-Z]+", ' ', str(text))
	tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
	text_tokens = tokenizer.tokenize(text)
	clean_text = []
	for word in text_tokens:
		if (word not in stopwords_english and word not in string.punctuation):
			stem_word = stemmer.stem(word)
			clean_text.append(stem_word)
	return clean_text
def converttimestamp(timestamp_string):
    obj = datetime.datetime.strptime(timestamp_string, '%Y-%m-%d')
    return obj

def calc_timedelta(ts1):
    ts1_obj = converttimestamp(ts1)
    ts2_obj = converttimestamp("2020-02-01")
    return (ts1_obj - ts2_obj).days
def calc_timestamp_withdelta(delta, delta_0):
    ts1_obj = converttimestamp(delta_0)
    timedelta_obj = datetime.timedelta(days = delta)
    ts2_obj = ts1_obj + timedelta_obj
    return ts2_obj.strftime("%Y-%m-%d")
def splitdict(string):
    i = 0
    dict_list = []
    for index in range(len(string) - 2):
        if string[index] == "}" and string[index+1] == "{" and string[index+2].isalpha() == False:
            sub_string = string[i:index+1]
            i = index + 1
            try:
                dict1 = json.loads(sub_string)
                dict_list.append(dict1)
            except Exception as e:
                dict1 = []
                #print(sub_string)
                print("ERROR!")
                print(e)
                pass
    sub_string = string[i:]
    dict2 = json.loads(sub_string)
    dict_list.append(dict2)
    return dict_list
def corpus_from_keyword_timeslice_tomoto(keyword):
    root_path = ""
    keyword_path = os.path.join(root_path, keyword)
    corpus = []
    time_slice = []
    distinct_dict = {}
    delta_0 = "2019-10-01"
    for i in range(0, 177):
        distinct_dict = {}
        # From day 0 to day 178
        dayfolder = calc_timestamp_withdelta(i, "2019-10-01")
        dayfolder_path = os.path.join(keyword_path, dayfolder)
        for file in os.listdir(dayfolder_path):
            with open(os.path.join(dayfolder_path, file), 'r', encoding = 'utf-8') as f:
                try:
                    listdict = splitdict(f.read())
                except Exception as e:
                    print(e)
                    listdict = []
                #DO WORK in 180 seconds for 100 tweets.
                try:
                    for each in listdict:
                        if "retweeted_status" in each.keys():
                            st = preprocessing(each['retweeted_status']["full_text"])
                        else:
                            st = preprocessing(each["full_text"])
                        if st in distinct_dict:
                            distinct_dict[st] += 1
                        else:
                            distinct_dict[st] = 1
                except Exception as e:
                    print(e)
        for each in distinct_dict:
            yield each, None, {'timepoint':i}
        print("Finished day " + str(i) + " ...with length " +str(len(distinct_dict)))
def corpus_separation(keyword, mdl_done):
    root_path = ""
    out_path = ""
    keyword_path = os.path.join(root_path, keyword)
    keyword_out_path = os.path.join(out_path, keyword)
    os.makedirs(keyword_out_path, exist_ok = True)
    topic_num = len(mdl.get_count_by_topics()[0])
    corpus = []
    time_slice = []
    distinct_dict = {}
    json_dict = {}
    json_dict_count = {}
    json_dict_output_num = {}
    for i in range(0, topic_num):
        json_dict_count[i] = 0
        json_dict[i] = []
        json_dict_output_num[i] = 0

    delta_0 = "2019-10-01"
    signal_a = False
           
    for i in range(0, 177):
        distinct_dict = {}
        # From day 0 to day 178
        dayfolder = calc_timestamp_withdelta(i, "2019-10-01")
        dayfolder_path = os.path.join(keyword_path, dayfolder)
        out_dayfolder_path = os.path.join(keyword_out_path, dayfolder)
        for file in os.listdir(dayfolder_path):
            print(file)
            with open(os.path.join(dayfolder_path, file), 'r', encoding = 'utf-8') as f:
                try:
                    listdict = splitdict(f.read())
                    #print(listdict)
                except Exception as e:
                    print(e)
                    listdict = []
                #DO WORK in 180 seconds for 100 tweets.
                for each in listdict:
                    try:
                        if "retweeted_status" in each.keys():
                            st = preprocessing(each['retweeted_status']["full_text"])
                            singal_a = False
                        else:
                            st = preprocessing(each["full_text"])
                            singal_a = True
                        if st in distinct_dict:
                            distinct_dict[st] += 1
                        else:
                            distinct_dict[st] = 1
                            if singal_a:
                                doc_inst = mdl_done.make_doc(preprocessing_to_token(each["full_text"]), timepoint = i)
                            else:
                                doc_inst = mdl_done.make_doc(preprocessing_to_token(each['retweeted_status']["full_text"]), timepoint = i)
                            topic_dist, ll = mdl_done.infer(doc_inst, iter = 1, parallel = 1)
                            #print(topic_dist)
                            topic_distribution_list = list(topic_dist)
                            for x in range(0, topic_num):
                                if topic_distribution_list[x] > 0.12:
                                    json_dict_count[x] += 1
                                    json_dict[x].append(each)
                                    if json_dict_count[x] == 100:
                                        json_dict_output_num[x] += 1
                                        outfile = str(json_dict_output_num[x]) + ".json"
                                        outfile_path = os.path.join(out_dayfolder_path, str(x))
                                        os.makedirs(outfile_path, exist_ok = True)
                                        outfile_path = os.path.join(outfile_path, outfile)
                                        with open(outfile_path, 'w', encoding = "utf-8") as w:
                                            while(len(json_dict[x]) > 0):
                                                content = json_dict[x].pop()
                                                w.write(json.dumps(content))
                                            w.close()
                                        json_dict_count[x] = 0
                    except Exception as e:
                        print(e)
                        continue
        print("Finished day " + str(i))
        for x in range(0, topic_num):
            outfile_path = os.path.join(out_dayfolder_path, str(x))
            os.makedirs(outfile_path, exist_ok = True)
            if json_dict_count[x] != 0:
                json_dict_output_num[x] += 1
                outfile = str(json_dict_output_num[x]) + ".json"
                final_path = os.path.join(outfile_path, outfile)
                with open(final_path, 'w', encoding = "utf-8") as w:
                    while(len(json_dict[x]) > 0):
                        content = json_dict[x].pop()
                        w.write(json.dumps(content))
                    w.close()
                json_dict_count[x] = 0
                json_dict_output_num[x] = 0
                
def corpus_separation_day(keyword, mdl_done, day):
    root_path = ""
    out_path = ""
    keyword_path = os.path.join(root_path, keyword)
    keyword_out_path = os.path.join(out_path, keyword)
    os.makedirs(keyword_out_path, exist_ok = True)
    topic_num = len(mdl.get_count_by_topics()[0])
    corpus = []
    time_slice = []
    distinct_dict = {}
    json_dict = {}
    json_dict_count = {}
    json_dict_output_num = {}
    for i in range(0, topic_num):
        json_dict_count[i] = 0
        json_dict[i] = []
        json_dict_output_num[i] = 0

    delta_0 = "2019-10-01"
    signal_a = False
    distinct_dict = {}
    # From day 0 to day 178
    dayfolder = calc_timestamp_withdelta(day, "2019-10-01")
    dayfolder_path = os.path.join(keyword_path, dayfolder)
    out_dayfolder_path = os.path.join(keyword_out_path, dayfolder)
    for file in os.listdir(dayfolder_path):
        print(file)
        with open(os.path.join(dayfolder_path, file), 'r', encoding = 'utf-8') as f:
            try:
                listdict = splitdict(f.read())
                #print(listdict)
            except Exception as e:
                print(e)
                listdict = []
            #DO WORK in 180 seconds for 100 tweets.
            for each in listdict:
                try:
                    if "retweeted_status" in each.keys():
                        st = preprocessing(each['retweeted_status']["full_text"])
                        singal_a = False
                    else:
                        st = preprocessing(each["full_text"])
                        singal_a = True
                    if st in distinct_dict:
                        distinct_dict[st] += 1
                        
                    else:
                        distinct_dict[st] = 1
                        if singal_a:
                            doc_inst = mdl_done.make_doc(preprocessing_to_token(each["full_text"]), timepoint = day)
                        else:
                            doc_inst = mdl_done.make_doc(preprocessing_to_token(each['retweeted_status']["full_text"]), timepoint = day)
                        topic_dist, ll = mdl_done.infer(doc_inst, iter = 1, parallel = 1)
                        #print(topic_dist)
                        topic_distribution_list = list(topic_dist)
                        for x in range(0, topic_num):
                            if topic_distribution_list[x] > 0.12:
                                json_dict_count[x] += 1
                                json_dict[x].append(each)
                                if json_dict_count[x] == 100:
                                    json_dict_output_num[x] += 1
                                    outfile = str(json_dict_output_num[x]) + ".json"
                                    outfile_path = os.path.join(out_dayfolder_path, str(x))
                                    os.makedirs(outfile_path, exist_ok = True)
                                    outfile_path = os.path.join(outfile_path, outfile)
                                    with open(outfile_path, 'w', encoding = "utf-8") as w:
                                        while(len(json_dict[x]) > 0):
                                            content = json_dict[x].pop()
                                            w.write(json.dumps(content))
                                        w.close()
                                    json_dict_count[x] = 0
                except Exception as e:
                    print(e)
                    continue
    print("Finished day " + str(day))
    for x in range(0, topic_num):
        outfile_path = os.path.join(out_dayfolder_path, str(x))
        os.makedirs(outfile_path, exist_ok = True)
        if json_dict_count[x] != 0:
            json_dict_output_num[x] += 1
            outfile = str(json_dict_output_num[x]) + ".json"
            final_path = os.path.join(outfile_path, outfile)
            with open(final_path, 'w', encoding = "utf-8") as w:
                while(len(json_dict[x]) > 0):
                    content = json_dict[x].pop()
                    w.write(json.dumps(content))
                w.close()
            json_dict_count[x] = 0
            json_dict_output_num[x] = 0
def corpus_check_stat(keyword, day):
    return 0
def corpus_separation_day_new(keyword, mdl_done, day):
    root_path = ""
    out_path = ""
    keyword_path = os.path.join(root_path, keyword)
    keyword_out_path = os.path.join(out_path, keyword)
    os.makedirs(keyword_out_path, exist_ok = True)
    topic_num = len(mdl.get_count_by_topics()[0])
    corpus = []
    time_slice = []
    distinct_dict = {}
    json_dict = {}
    json_dict_count = {}
    json_dict_output_num = {}
    for i in range(0, topic_num):
        json_dict_count[i] = 0
        json_dict[i] = []
        json_dict_output_num[i] = 0

    delta_0 = "2019-10-01"
    signal_a = False
    distinct_dict = {}
    # From day 0 to day 178
    dayfolder = calc_timestamp_withdelta(day, "2019-10-01")
    dayfolder_path = os.path.join(keyword_path, dayfolder)
    out_dayfolder_path = os.path.join(keyword_out_path, dayfolder)
    count = 0
    for file in os.listdir(dayfolder_path):
        print(file)
        with open(os.path.join(dayfolder_path, file), 'r', encoding = 'utf-8') as f:
            try:
                listdict = splitdict(f.read())
                #print(listdict)
            except Exception as e:
                print(e)
                listdict = []
            #DO WORK in 180 seconds for 100 tweets.
            for each in listdict:
                try:
                    if "retweeted_status" in each.keys():
                        st = preprocessing(each['retweeted_status']["full_text"])
                        singal_a = False
                    else:
                        st = preprocessing(each["full_text"])
                        singal_a = True
                    if st in distinct_dict:
                        #distinct_dict[st] += 1
                        #if already in, we read the distinct_dict[st] which is a list of numbers storing the distribution_list of topic.
                        topic_distribution_list = distinct_dict[st]
                    else:
                        #distinct_dict[st] = 1
                        if singal_a:
                            doc_inst = mdl_done.make_doc(preprocessing_to_token(each["full_text"]), timepoint = day)
                        else:
                            doc_inst = mdl_done.make_doc(preprocessing_to_token(each['retweeted_status']["full_text"]), timepoint = day)
                        topic_dist, ll = mdl_done.infer(doc_inst, iter = 1, parallel = 1)
                        #print(topic_dist)
                        topic_distribution_list = list(topic_dist)
                        distinct_dict[st] = topic_distribution_list                    
                    for x in range(0, topic_num):
                        if topic_distribution_list[x] > 0.12:
                            json_dict_count[x] += 1
                            count += 1
                            json_dict[x].append(each)
                            if json_dict_count[x] == 100:
                                json_dict_output_num[x] += 1
                                outfile = str(json_dict_output_num[x]) + ".json"
                                outfile_path = os.path.join(out_dayfolder_path, str(x))
                                os.makedirs(outfile_path, exist_ok = True)
                                outfile_path = os.path.join(outfile_path, outfile)
                                with open(outfile_path, 'w', encoding = "utf-8") as w:
                                    while(len(json_dict[x]) > 0):
                                        content = json_dict[x].pop()
                                        w.write(json.dumps(content))
                                    w.close()
                                json_dict_count[x] = 0
                except Exception as e:
                    print(e)
                    continue
    print("Finished day " + str(day) + " ... with count " + str(count))
    for x in range(0, topic_num):
        outfile_path = os.path.join(out_dayfolder_path, str(x))
        os.makedirs(outfile_path, exist_ok = True)
        if json_dict_count[x] != 0:
            json_dict_output_num[x] += 1
            outfile = str(json_dict_output_num[x]) + ".json"
            final_path = os.path.join(outfile_path, outfile)
            with open(final_path, 'w', encoding = "utf-8") as w:
                while(len(json_dict[x]) > 0):
                    content = json_dict[x].pop()
                    w.write(json.dumps(content))
                w.close()
            json_dict_count[x] = 0
            json_dict_output_num[x] = 0
def chunking(lst, n):
    '''
    Given lst,
    return a list of object in lst with max length of n
    '''
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
keyword = "africa"
porter_stemmer = nltk.PorterStemmer().stem
corpus2 = tp.utils.Corpus(
    tokenizer=tp.utils.SimpleTokenizer(porter_stemmer)
)
corpus2.process(corpus_from_keyword_timeslice_tomoto(keyword))

mdl = tp.DTModel(min_cf=100,min_df = 70, k=25, t=177, corpus=corpus2)
mdl.train(0)

print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(mdl.docs), len(mdl.used_vocabs), mdl.num_words
))
print('Removed Top words: ', *mdl.removed_top_words)

# Let's train the model
for i in range(0, 1000, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)
print('Iteration: {:04}, LL per word: {:.4}'.format(1000, mdl.ll_per_word))

mdl.summary()
mdl.save("DTM-mdl-" + keyword + ".save")
topic_dist_by_time = np.zeros(shape=[mdl.num_timepoints, mdl.k], dtype=np.float)
for doc in mdl.docs:
    topic_dist_by_time[doc.timepoint] += doc.get_topic_dist()

topic_dist_by_time /= mdl.num_docs_by_timepoint[:, np.newaxis]

for k in range(mdl.k):
    print('Topic #{}'.format(k), *(w for w, _ in mdl.get_topic_words(k, 0, top_n=5)))
    print(topic_dist_by_time[:, k])

for timepoint in range(mdl.num_timepoints):
    topic_term_dists = np.stack([mdl.get_topic_word_dist(k, timepoint=timepoint) for k in range(mdl.k)])
    doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs if doc.timepoint == timepoint])
    doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
    doc_lengths = np.array([len(doc.words) for doc in mdl.docs if doc.timepoint == timepoint])
    vocab = list(mdl.used_vocabs)
    term_frequency = mdl.used_vocab_freq

    prepared_data = pyLDAvis.prepare(
        topic_term_dists, 
        doc_topic_dists, 
        doc_lengths, 
        vocab, 
        term_frequency,
        start_index=0,
        sort_topics=False
    )
    #pyLDAvis.save_html(prepared_data, 'dtmvis_outbreak_{}.html'.format(timepoint))

In [ ]:
import multiprocessing
process_list = []
for each in range(0, 177):
    st = multiprocessing.Process(target = corpus_separation_day_new, args = (keyword, mdl, each))
    process_list.append(st)
for each in chunking(process_list, 30):
    for each_process in each:
        each_process.start()
    for each_process in each:
        each_process.join()